# Hyetograph and Hydrograph of event

### For the function to work, arrays of flow and rain should be given using as indeces daytime format.
#### Examle 1:

In [ ]:
flow = pd.read_csv('week_8_2003-2013_flow_data_15_min_corr.csv')
flow.index = pd.to_datetime(flow.loc[:, ['year', 'month', 'day', 'hour', 'minute']])
rain.index = pd.to_datetime(rain.loc[:, ['year', 'month', 'day', 'hour', 'minute']]).rename(columns = {'02146409': '409', '02146507': '507'})
rain = rain.drop(axis = 1, columns = ['year', 'month', 'day', 'hour', 'minute']).rename(columns = {'02146409': '409', '02146507': '507'})
rain_24h = pd.DataFrame()
rain_24h['409sum'] = rain['409'].resample(rule = 'D').sum()
rain_24h['507sum'] = rain['507'].resample(rule = 'D').sum() 
rain_24h['409mean'] = rain['409'].resample(rule = 'D').mean() 
rain_24h['507mean'] = rain['507'].resample(rule = 'D').mean()

flow_24h = pd.DataFrame()
flow_24h['409sum'] = summer_flow['409mmh'].resample(rule = 'D').sum()
flow_24h['507sum'] = summer_flow['507mmh'].resample(rule = 'D').sum()
flow_24h['409mean'] = summer_flow['409mmh'].resample(rule = 'D').mean()
flow_24h['507mean'] = summer_flow['507mmh'].resample(rule = 'D').mean()
flow_24h.head()

#### Examle 2:

In [ ]:
charlotte_rainfall = pd.read_csv(r'charlotte_rg_2003-2014.csv', names = [ 'year' , 'month' , 'day' , 'hour' , 'minute', 'rainfall'])
charlotte_rainfall.loc[:, 'dt'] = pd.to_datetime(charlotte_rainfall.loc[:, ['year', 'month', 'day', 'hour', 'minute']])
charlotte_rainfall = charlotte_rainfall.set_index('dt')

#MILITARY HOURS TO DATETIME
rotterdam_rainfall = pd.read_csv(r'rotterdam_rg_2003-2014.csv', skipinitialspace=True)

#Format data for the Rotterdam data set
dates = rotterdam_rainfall['Datum']
time = rotterdam_rainfall['Tijd']
dates = dates.map(str)
date_split = dates.str.extract('(.{4})(.{2})(.{2})', expand=True)

time=time.apply(lambda x: '%04d' % x)
time_split = time.str.extract('(.{2})(.{2})', expand=True)

date_split.loc[:,3] = time_split.loc[:,0]
date_split.loc[:,4] = time_split.loc[:,1]

rotterdam_rainfall.loc[:,'dt'] = pd.to_datetime(dict(year=date_split[0], month=date_split[1], day=date_split[2], hour=date_split[3], minute=date_split[4]))
rotterdam_rainfall.index=rotterdam_rainfall['dt']

# Hyetograph and Hydrograph of event

In [1]:
def hyetoplot(down, up, avalaible_dates, plotsize, downscale, upscale, numberofdates):
    fig, ax = plt.subplots(figsize = plotsize)
    ax.plot(avalaible_dates, down, "black", zorder = 2)
    ax2 = ax.twinx()
    ax2.bar(avalaible_dates, height = up, label = 'Rainfall Event', width = 2, zorder = 2)
    ax2.set_ylim([np.min(up), upscale * np.max(up)])
    ax.set_ylim([np.max(down), downscale * np.min(down)])
    ax.set_xlabel('Date', fontsize=15)
    ax.set_ylabel('Backscatter Sigma0', fontsize=15)
    ax2.set_ylabel('Rainfall [mm/day]', fontsize=15)
    ax2.invert_yaxis()
    plt.grid()
    fig.autofmt_xdate()
    ax.xaxis.set_major_locator(plt.MaxNLocator(numberofdates))
    ax.fill_between(avalaible_dates, down, color ='green', zorder = 2, label='Sigma0')
#     for i, j in enumerate(up):
#         if j > 0:
#             ax.axvline(avalaible_dates[i], down[i], color="grey", lw = 3, ls = '--', alpha = 0.8, zorder = 1)
    plt.grid([])
#     ax.axvline(0, 0, color="grey", lw = 3, ls = '--', alpha = 0.8, zorder = 1, label = 'Projection of Rainfall event on Sigma0')
    ax.tick_params(labelsize = 15)
    ax2.tick_params(labelsize = 15)
    handles,labels = [],[]
    for ax in fig.axes:
        for h,l in zip(*ax.get_legend_handles_labels()):
            handles.append(h)
            labels.append(l)
    plt.legend(handles, labels, loc = 'best', fontsize = 15)
    plt.show();